## Tokenization using Byte-Pair Encoding and a Unigram Language Model

Author: Pierre Nugues with help from Marcus Klang

In this assignment, you will create a tokenization program to handle subwords.

In many scripts from Asia, like Chinese, Korean, or Japanese scripts, tokenization cannot rely on white spaces. The byte-pair encoding and the unigram language model are techniques that are now common in machine translation to carry out a tokenization at a subword level. Subword level tokenization shows better multilingual capabilities.

You will follow two papers: 
* Subword Regularization: _Improving Neural Network Translation Models with Multiple Subword Candidates_ by Kudo (2018) (https://arxiv.org/pdf/1804.10959.pdf) and 
* _Byte Pair Encoding is Suboptimal for Language Model Pretraining_ by Bostrom and Durrett (2020) (https://aclanthology.org/2020.findings-emnlp.414.pdf). 

In addition, you will start from a clear and easy-to-understand description in Google’s Neural Machine Translation System: _Bridging the Gap between Human and Machine Translation_ by Wu et al. (2016). (Do not read them now)
https://arxiv.org/abs/1609.08144

You will use a small corpus make it easier to test and correct your code. Note also that you will use _characters_ and not _bytes_ in this lab as this is simpler to implement. For a complete program, see the link at the end.

**In your report, be sure to answer all the questions. Please reuse the section titles of this notebook so that I can check your answers more easily**

## Preliminaries

As an overall description of the subword tokenizers, read Sections 4 (introduction paragraph) and 4.1. in the paper on translation: _Bridging the Gap between Human and Machine Translation_ by Wu et al. (2016), https://arxiv.org/abs/1609.08144.  

In your report, in a few lines (10 to 15 lines or so) you will:

1. Outline the difference with tokenization as you saw it during the course;
2. Imagine how the tokens will be learned (this will developed in the rest of the lab);
3. Summarize what could be the advantages for Asian languages, unknown words, and translation.

Commenting Sections 4 and 4.1 in your report is **mandatory**. If you are curious, you can read the complete article.

## Understanding SentencePiece
SentencePiece is the combination of BPE and a language model. To be sure you understand how it works, you will first run this code.
These are the first cells from a larger program: https://github.com/google/sentencepiece/blob/master/python/sentencepiece_python_module_example.ipynb
from Google.


In [2]:
%pip install sentencepiece
!wget https: // raw.githubusercontent.com/google/sentencepiece/master/data/botchan.txt


Note: you may need to restart the kernel to use updated packages.
--2023-09-16 14:25:10--  ftp://https/
           => ‘.listing’
Resolving https (https)... failed: Name or service not known.
wget: unable to resolve host address ‘https’
//: Scheme missing.
URL transformed to HTTPS due to an HSTS policy
--2023-09-16 14:25:10--  https://raw.githubusercontent.com/google/sentencepiece/master/data/botchan.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 278779 (272K) [text/plain]
Saving to: ‘botchan.txt.4’

botchan.txt.4       100%[===================>] 272.25K  --.-KB/s    in 0.04s   

2023-09-16 14:25:10 (7.39 MB/s) - ‘botchan.txt.4’ saved [278779/278779]

FINISHED --2023-09-16 14:25:10--
Total wall clock time: 0.1s
Downloaded: 1 files, 272K in 0.04s (7.39 MB

In [3]:
import sentencepiece as spm


Train a model from a corpus. Once trained, read the content of `m.vocab`. Be sure to undestand its content.

In [4]:
spm.SentencePieceTrainer.train(
    '--input=botchan.txt --model_prefix=m --vocab_size=2000')


sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=botchan.txt --model_prefix=m --vocab_size=2000
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: botchan.txt
  input_format: 
  model_prefix: m
  model_type: UNIGRAM
  vocab_size: 2000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  p

In [5]:
sp = spm.SentencePieceProcessor()
sp.load('m.model')


True

In [6]:
print(sp.encode('Tokenization using Byte-Pair Encoding'))


[386, 315, 27, 52, 1997, 227, 282, 14, 1058, 237, 22, 717, 1060, 443, 27, 67, 38, 20, 14]


In [7]:
sp.id_to_piece(386)


'▁To'

In [8]:
[sp.id_to_piece(i) for i in range(20)]


['<unk>',
 '<s>',
 '</s>',
 ',',
 '.',
 '▁the',
 '▁I',
 's',
 '▁to',
 '▁a',
 '▁and',
 '▁of',
 '▁',
 'ed',
 'ing',
 '▁in',
 '▁was',
 '▁"',
 '▁it',
 't']

In [9]:
print(sp.encode('Tokenization using Byte-Pair Encoding', out_type=str))


['▁To', 'ke', 'n', 'i', 'z', 'ation', '▁us', 'ing', '▁By', 'te', '-', 'P', 'air', '▁E', 'n', 'c', 'o', 'd', 'ing']


## Design of the BPE Algorithm

The first algorithm to build the subwords from a corpus is a byte-pair encoding (BPE), due to Gage (1994). In the lab, you will first read two sections of more recent articles as they are easier to understand and specifically targeted to natural language processing.

Read these two sections:

1. Section 3.1 of _Subword Regularization: Improving Neural Network Translation Models with Multiple Subword Candidates_ (https://arxiv.org/pdf/1804.10959.pdf) by Kudo (2018).
2. Section 2, algorithm 1 of _Byte Pair Encoding is Suboptimal for Language Model Pretraining_ (https://aclanthology.org/2020.findings-emnlp.414.pdf) by Bostrom and Durrett (2020).

In your report, **summarize** (10 to 15 lines or so) with your own words the byte-pair encoding (BPE) algorithm as described by Kudo (2018) and Bostrom and Durrett (2020) (Only BPE and not the unigram language model).

## BPE Programming

You will now program a byte-pair encoding program in Python. You will do it step by step. The first part will be to extract the subwords from a corpus. Note that you will use the characters, not the bytes.

In [10]:
import regex as re
import tqdm as tqdm
import math


First use a small corpus and then, if you have time, test your program on a larger one. Here we take the smallest novel from Selma Lagerlöf in our corpus.

In [11]:
import os
from zipfile import ZipFile
import requests

# Parameters for Selma dataset
SELMA_URL = "https://github.com/pnugues/ilppp/raw/master/programs/corpus/Selma.zip"

SELMA_FILES = [
    os.path.join("Selma", fname)
    for fname in
    [
        "bannlyst.txt",
        "gosta.txt",
        "herrgard.txt",
        "jerusalem.txt",
        "kejsaren.txt",
        "marbacka.txt",
        "nils.txt",
        "osynliga.txt",
        "troll.txt"
    ]
]


def download_and_extract_selma():
    """Downloads and unpacks Selma.zip"""

    # Download if not all files exist
    req = requests.get(SELMA_URL, stream=True)
    if req.status_code != 200:
        print("Failed to download file, got status: " + req.status_code)
        req.close()
    else:
        with open("Selma.zip", "wb") as fd:
            written = 0
            for chunk in req.iter_content(chunk_size=65536):
                fd.write(chunk)
                written += len(chunk)
                print("Downloading: %d bytes written to Selma.zip" % written)

        print("Selma.zip donwnloaded.")
        req.close()

        selma_zipfile = ZipFile("Selma.zip")
        selma_files_to_extract = [zi for zi in selma_zipfile.filelist if not zi.filename.startswith(
            "__") and zi.filename.endswith(".txt")]
        for zi in selma_files_to_extract:
            selma_zipfile.extract(zi)
            print("Extracted: " + zi.filename)

        print("Done!")


# If not all path exists (all are true), then download
if not all([os.path.exists(fname) for fname in SELMA_FILES]):
    download_and_extract_selma()
else:
    print("Selma has been downloaded.")

SELMA_FILES


Selma has been downloaded.


['Selma/bannlyst.txt',
 'Selma/gosta.txt',
 'Selma/herrgard.txt',
 'Selma/jerusalem.txt',
 'Selma/kejsaren.txt',
 'Selma/marbacka.txt',
 'Selma/nils.txt',
 'Selma/osynliga.txt',
 'Selma/troll.txt']

In [12]:
#FILE_PATH = '../../corpus/Selma.txt'
FILE_PATH = 'Selma/herrgard.txt'


Read the corpus and store it in the `corpus` string variable.

In [13]:
with open(FILE_PATH, encoding='utf8') as f:
    corpus = f.read().strip()


Replace all the space sequences in `corpus`, including newlines and tabulations, and normalize them as one space. Use the `\s` class.

In [14]:
# Write your code
corpus = re.sub(r'[\s]+', ' ', corpus)


In [15]:
corpus[:100]


'Selma Lagerlöf En herrgårdssägen Bokutgåva Albert Bonniers förlag, Stockholm 1899. I. Det var en skö'

### BPE

#### Initial Vocabulary

Write the code (one instruction) to split the corpus in a list of characters and store the results in `corpus_l`. This is just a type conversion. Given the input:
<pre><span style="font-size: 12pt;">corpus = 'De senaste fem &aring;ren har cirka 25 000 unga'</span></pre>

Return:
<pre><span style="font-size: 12pt;">corpus_l = ['D', 'e', ' ', 's', 'e', 'n', 'a', 's', 't', 'e', ' ', 'f', 'e', 'm', ' ', ...]</span></pre>

In [16]:
# Write your code
corpus_l = list(corpus)


In [17]:
corpus_l[:15]


['S', 'e', 'l', 'm', 'a', ' ', 'L', 'a', 'g', 'e', 'r', 'l', 'ö', 'f', ' ']

Extract the set of characters that will serve as initial subword tokens:

1. Write a statement to extract the set of all the characters from `corpus_l`; 
2. Exclude the space from this set and call the resulting set: `char_set`.

In [18]:
# Write your code
char_set = set(corpus_l)
char_set.remove(' ')


In [19]:
print(sorted(char_set))

['!', ',', '-', '.', '1', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'X', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z', '»', 'Ä', 'Å', 'Ö', 'ä', 'å', 'é', 'ö', '–', '’']


In [20]:
len(char_set)


67

Using code from the previous question, write an `initial_vocabulary()` function taking the the `corpus_l` variable as input and returning the the set of all characters appearing in the corpus (the initial character set), deprived from the white space.

In [21]:
# Write your code here
def initial_vocabulary(corpus_l):
    vocabulary = set(corpus_l)
    vocabulary.remove(' ')
    return vocabulary


In [22]:
initial_vocabulary(corpus_l)


{'!',
 ',',
 '-',
 '.',
 '1',
 '8',
 '9',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'R',
 'S',
 'T',
 'U',
 'V',
 'X',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'r',
 's',
 't',
 'u',
 'v',
 'x',
 'y',
 'z',
 '»',
 'Ä',
 'Å',
 'Ö',
 'ä',
 'å',
 'é',
 'ö',
 '–',
 '’'}

#### Counting

Write a `pair_count()` function that takes a list of tokens as input, possibly single characters or subword tokens, and that counts the adjacent pairs (bigrams). You will implement these counts as dictionaries: The key will be a pair (tuple) of adjacent symbols and the value, its frequency. Remember that you cannot cross whitespaces, i.e. a pair cannot include a whitespace.

Given the input

`['D', 'e', ' ', 's', 'e', 'n', 'a', 's', 't', ...]`
count_pairs should return a dictionary: 


`{('D', 'e'): 1, ('s', 'e'): 1, ('e', 'n'): 1, ('n', 'a'): 1, ...}`

In [23]:
# Write your code here
def pair_count(corpus_l: list[str]):
    corpus_l = list(filter(lambda x: x != ' ', corpus_l))
    pairs = {}
    for i in range(len(corpus_l) - 1):
        pair = (corpus_l[i], corpus_l[i + 1])
        if pair in pairs:
            pairs[pair] += 1
        else:
            pairs[pair] = 1
    return pairs


In [24]:
pairs = pair_count(corpus_l)


In [25]:
pairs


{('S', 'e'): 7,
 ('e', 'l'): 592,
 ('l', 'm'): 45,
 ('m', 'a'): 501,
 ('a', 'L'): 1,
 ('L', 'a'): 8,
 ('a', 'g'): 572,
 ('g', 'e'): 711,
 ('e', 'r'): 1330,
 ('r', 'l'): 273,
 ('l', 'ö'): 81,
 ('ö', 'f'): 217,
 ('f', 'E'): 1,
 ('E', 'n'): 21,
 ('n', 'h'): 564,
 ('h', 'e'): 809,
 ('r', 'r'): 219,
 ('r', 'g'): 197,
 ('g', 'å'): 343,
 ('å', 'r'): 318,
 ('r', 'd'): 735,
 ('d', 's'): 205,
 ('s', 's'): 291,
 ('s', 'ä'): 213,
 ('ä', 'g'): 143,
 ('e', 'n'): 3520,
 ('n', 'B'): 1,
 ('B', 'o'): 2,
 ('o', 'k'): 56,
 ('k', 'u'): 573,
 ('u', 't'): 314,
 ('t', 'g'): 189,
 ('å', 'v'): 71,
 ('v', 'a'): 1426,
 ('a', 'A'): 3,
 ('A', 'l'): 24,
 ('l', 'b'): 60,
 ('b', 'e'): 314,
 ('r', 't'): 460,
 ('t', 'B'): 1,
 ('o', 'n'): 1610,
 ('n', 'n'): 1167,
 ('n', 'i'): 368,
 ('i', 'e'): 71,
 ('r', 's'): 516,
 ('s', 'f'): 95,
 ('f', 'ö'): 980,
 ('ö', 'r'): 1317,
 ('l', 'a'): 905,
 ('g', ','): 192,
 (',', 'S'): 1,
 ('S', 't'): 88,
 ('t', 'o'): 588,
 ('o', 'c'): 1223,
 ('c', 'k'): 794,
 ('k', 'h'): 60,
 ('h', 'o'): 1

Determine the most frequent pair

In [26]:
# write your code
most_freq_pair = max(pairs, key=pairs.get)


In [27]:
most_freq_pair


('d', 'e')

In [28]:
''.join(most_freq_pair)


'de'

#### The First Iteration

We store the initial symbols in a `vocabulary` variable

In [29]:
vocabulary = initial_vocabulary(corpus_l)


In [30]:
len(vocabulary)


67

Add your most frequent pair to the vocabulary after one iteration as well as the merge operations: `merge_ops`. `merge_ops` will contain the merge operations in the order you created them.

In [31]:
merge_ops = []


In [75]:
# write your code here
most_freq_pair = max(pairs, key=pairs.get)
merge_ops.append(most_freq_pair)
vocabulary.add(''.join(most_freq_pair))


In [33]:
len(vocabulary)


68

In [77]:
merge_ops


[('d', 'e'),
 ('e', 'n'),
 ('a', 'n'),
 ('t', 't'),
 ('a', 'r'),
 ('s', 't'),
 ('o', 'm'),
 ('o', 'n'),
 ('l', 'l'),
 ('ö', 'r'),
 ('a', 'tt'),
 ('a', 'de'),
 ('c', 'h'),
 ('i', 'g'),
 ('n', 'g'),
 ('e', 'r'),
 ('o', 'ch'),
 ('v', 'ar'),
 ('h', 'on'),
 ('e', 't'),
 ('f', 'ör'),
 ('s', 'k'),
 ('ä', 'r'),
 ('c', 'k'),
 ('h', 'an'),
 ('o', 'r'),
 ('n', 'a'),
 ('de', 't'),
 ('s', 'å'),
 ('e', 'j'),
 ('ä', 'n'),
 ('i', 'n'),
 ('u', 'n'),
 ('de', 'n'),
 ('a', 'g'),
 ('e', 'd'),
 ('s', 'om'),
 ('i', 'll'),
 ('f', 'v'),
 ('p', 'å'),
 ('en', 'n'),
 ('i', 'd'),
 ('a', 'm'),
 ('l', 'i'),
 ('h', 'enn'),
 ('f', 'r'),
 ('henn', 'e'),
 ('h', 'ade'),
 ('a', 'll'),
 ('i', 'ng'),
 ('d', 'e')]

#### Incremental Construction
We will now incrementally build the vocabulary.

Create a `merge_bigrams()` function that takes a list of tokens, `corpus_l`, and a pair of subword tokens `(token_r, token_l)` as input and merges adjacent sequences token_r, token_l into a new token, `token_new`, replacing the sequence `token_r, token_l` in `corpus_l`. Your function will return a new list. 

Given the input 

`corpus_l = ['D', 'e', ' ', 's', 'e', 'n', 'a', 's', 't', ...]`

`merge_bigrams(corpus_l, ('e', 'n'))` should return where all the seuquences of 'e' and 'n' have been merged:

`['D', 'e', ' ', 's', 'en', 'a', 's', 't', ...]`

And reapplying `merge_bigrams(corpus_l, ('s', 'en'))` to this corpus should return

`['D', 'e', ' ', 'sen', 'a', 's', 't', ...]`

You will apply a greedy algorithm. Given the pair ('a', 'a') and the list ['a', 'a', 'a'], the result will be: ['aa', 'a']

In [35]:
# Write your code here
def merge_bigrams(corpus_l, pair):
    new_corpus_l = []
    skip = False
    skip_next = False
    for i in range(len(corpus_l)):
        current = corpus_l[i]
        next_char = corpus_l[i + 1] if i < len(corpus_l) - 1 else None
        next_next_char = corpus_l[i + 2] if i < len(corpus_l) - 2 else None
        if skip:
            skip = False
        elif skip_next:
            skip_next = False
        elif next_char == " " and (current, next_next_char) == pair:
            new_corpus_l.append(''.join(pair))
            skip_next = True
            skip = True
        elif (current, next_char) == pair:
            new_corpus_l.append(''.join(pair))
            skip = True
        else:
            new_corpus_l.append(corpus_l[i])
    return new_corpus_l


In [36]:
corpus_test = ['D', 'e', ' ', 's', 'e', 'n', 'a', 's', 't']
merge_bigrams(corpus_test, ('e', 'n'))


['D', 'e', ' ', 's', 'en', 'a', 's', 't']

In [37]:
merge_bigrams(merge_bigrams(corpus_test, ('e', 'n')), ('s', 'en'))


['D', 'e', ' ', 'sen', 'a', 's', 't']

#### Byte Pair Encoding (BPE): Building the Vocabulary

Write now a `BPE()` function following Algorithm 1 in _Byte Pair Encoding is Suboptimal for Language Model Pretraining_ by Bostrom and Durrett (2020). 

Your function will take `corpus_l` and the vocabulary size `k` as input. This size `k` will correspond to the count of new subwords added to the initial list of symbols. With your initial corpus, you should have 67 found symbols. With `k = 10`, you will add 10 subwords to this initial list. Note that Bostrom and Durrett (2020) define their $k_\text{Bostrom and Durrett}$ as `k + initial vocabulary`. 

Return the vocabulary of subword tokens in the form of a list, the initial vocabulary and the subwords you will create, as well as the merge operations in the order you created them

You will start from the initial vocabulary and `k` will be the number of symbols you add to this vocabulary.

In [74]:
# Write your code here
def BPE(corpus_l, k):
    initial_vocab = initial_vocabulary(corpus_l)
    vocabulary = initial_vocabulary(corpus_l)
    merge_ops = []
    while len(vocabulary) < k + len(initial_vocab):
        pairs = pair_count(corpus_l)
        most_freq_pair = max(pairs, key=pairs.get)
        while not re.fullmatch(r'[\p{L}]+', ''.join(most_freq_pair)):
            del pairs[most_freq_pair]
            most_freq_pair = max(pairs, key=pairs.get)
        merge_ops.append(most_freq_pair)
        vocabulary.add(''.join(most_freq_pair))
        corpus_l = merge_bigrams(corpus_l, most_freq_pair)
    ...
    return vocabulary, merge_ops


We build a vocabulary of 50 subwords in addition to our initial set of symbols

In [39]:
vocabulary, merge_ops = BPE(corpus_l, 50)


de
en
an
tt
ar
st
om
on
ll
ör
att
ade
ch
ig
ng
er
och
var
hon
et
för
sk
är
ck
han
or
na
det
så
ej
än
in
un
den
ag
ed


som
ill
fv
på
enn
id
am
li
henn
fr
henne
hade
all
ing


In [40]:
len(vocabulary), len(merge_ops)

(117, 50)

In [41]:
print(vocabulary)


{'B', 'é', 'K', 'henn', 'T', 'on', ';', 'g', 'ar', 'b', 'or', 'M', '.', '9', 'll', 'och', 'ed', 'å', 'P', 'r', 'V', 'Å', 'am', '8', '’', 'd', 'O', 'y', 'enn', 'ng', 'är', 'a', '!', 'ill', 'än', 'om', 'J', 'v', 'et', 'G', 'I', 'i', '?', 'u', 'X', 'Ä', 'han', 'ck', 'D', 'S', 'en', 'hade', 'ör', 'E', 'all', 'j', 'ch', 's', 'an', 'att', '_', 'den', 'na', 'på', 'ing', 'hon', 'k', 'U', 't', 'fr', 'de', 'som', 'f', 'A', 'l', 'x', 'ä', '–', 'un', 'var', 'ö', 'Ö', 'st', 'in', 'li', ':', 'C', 'h', '1', 'ag', 'H', 'c', 'N', 'så', ',', '-', 'ade', 'fv', 'id', 'henne', 'p', 'm', 'e', 'för', 'det', 'R', 'o', 'sk', 'tt', '»', 'n', 'er', 'F', 'ig', 'z', 'L', 'ej'}


In [42]:
print(merge_ops)


[('d', 'e'), ('e', 'n'), ('a', 'n'), ('t', 't'), ('a', 'r'), ('s', 't'), ('o', 'm'), ('o', 'n'), ('l', 'l'), ('ö', 'r'), ('a', 'tt'), ('a', 'de'), ('c', 'h'), ('i', 'g'), ('n', 'g'), ('e', 'r'), ('o', 'ch'), ('v', 'ar'), ('h', 'on'), ('e', 't'), ('f', 'ör'), ('s', 'k'), ('ä', 'r'), ('c', 'k'), ('h', 'an'), ('o', 'r'), ('n', 'a'), ('de', 't'), ('s', 'å'), ('e', 'j'), ('ä', 'n'), ('i', 'n'), ('u', 'n'), ('de', 'n'), ('a', 'g'), ('e', 'd'), ('s', 'om'), ('i', 'll'), ('f', 'v'), ('p', 'å'), ('en', 'n'), ('i', 'd'), ('a', 'm'), ('l', 'i'), ('h', 'enn'), ('f', 'r'), ('henn', 'e'), ('h', 'ade'), ('a', 'll'), ('i', 'ng')]


#### BPE Tokenizer

You will now use the vocabulary you obtained to tokenize a text stored in the corpus string.

You will apply the merge operations in the same order you created them. You will call this function `tokenize_bpe()` that will take two inputs: `corpus` and `merge_ops`, and that will return the tokenized text in the form of a list.

    def tokenize_bpe(corpus, merge_ops):

      ...

      return tokens

This function is easy. Just reuse the `merge_bigrams()` function and a loop

In [43]:
# Write your code here
def tokenize_bpe(corpus, merge_ops):
    corpus_l = list(corpus)
    for merge_op in merge_ops:
        corpus_l = merge_bigrams(corpus_l, merge_op)
    return corpus_l


In [44]:
print(tokenize_bpe(corpus, merge_ops)[:200])


['S', 'e', 'l', 'm', 'a', ' ', 'L', 'ag', 'er', 'l', 'ö', 'f', ' ', 'E', 'n', ' ', 'h', 'er', 'r', 'g', 'å', 'r', 'd', 's', 's', 'ä', 'g', 'en', ' ', 'B', 'o', 'k', 'u', 't', 'g', 'å', 'v', 'a', ' ', 'A', 'l', 'b', 'er', 't', ' ', 'B', 'on', 'n', 'i', 'er', 's', ' ', 'för', 'l', 'ag', ',', ' ', 'S', 't', 'o', 'ck', 'h', 'o', 'l', 'm', ' ', '1', '8', '9', '9', '.', ' ', 'I', '.', ' ', 'D', 'et', ' ', 'var', ' ', 'en', ' ', 'sk', 'ö', 'n', ' ', 'h', 'ö', 'st', 'd', 'ag', ' ', 'h', 'än', 'e', 'm', 'o', 't', ' ', 's', 'l', 'u', 't', 'et', ' ', 'a', 'f', ' ', 't', 'r', 'e', 'tt', 'i', 'o', 't', 'a', 'l', 'et', '.', ' ', 'P', 'å', ' ', 'den', ' ', 't', 'i', 'den', ' ', 'f', 'an', 'n', 's', ' ', 'i', ' ', 'U', 'p', 's', 'a', 'l', 'a', ' ', 'e', 'tt', ' ', 'h', 'ö', 'g', 't', ',', ' ', 'g', 'u', 'l', 'tt', 'v', 'å', 'v', 'å', 'n', 'ing', 's', 'h', 'u', 's', ',', ' ', 'som', ' ', 'st', 'o', 'd', ' ', 'un', 'de', 'r', 'l', 'ig', 't', ' ', 'en', 's', 'am', 't', ' ', 'på', ' ', 'en', ' ', 'li', 't

## Unigram Language Model

You are now done with BPE and you can now consider the unigram language model.

Read these two sections:

1. Section 3.2 of _Subword Regularization: Improving Neural Network Translation Models with Multiple Subword Candidates_ (https://arxiv.org/pdf/1804.10959.pdf) by Kudo (2018).
2. Section 2, algorithm 2 and the related text of _Byte Pair Encoding is Suboptimal for Language Model Pretraining_ (https://aclanthology.org/2020.findings-emnlp.414.pdf) by Bostrom and Durrett (2020).

In your report, **summarize** (10 to 15 lines or so) with your own words the tokenization with a unigram language model as described by by Kudo (2018) and Bostrom and Durrett (2020). You will notably consider two aspects:
1. How to obtain the subword vocabulary;
2. How to tokenize a text.

In your report, given what you have done on the byte-pair encoding, how would you build the “reasonably big seed vocabulary” needed for the unigram language model?

### Unigram Probabilities

Starting from the “reasonably big seed vocabulary”, you will now fit a unigram language model. You will start with a vocabulary of 50 subwords in addition to the character set and reduce it to 49, i.e. you will find one subword to discard.

Kudo (2018) proposes the expectation-maximization algorithm. We will ignore this step. Instead, in this lab, you will approximate the language model with the BPE algorithm.

Write a `unigram_lm()` function that takes a corpus string and a vocabulary of subword tokens as input and returns a dictionary, where the keys are the subwords and each key value, the key relative frequency:

    def unigram_lm(corpus, vocabulary):

       ...

      return unigram_probs
Your function will:

1. Tokenize your corpus with BPE (you can reuse the `tokenize_bpe()` function);
2. Estimate the probability of each word (simply count the occurrences of the subwords and divide them by the length of the tokenized corpus);
3. Return this model as a dictionary.

In [45]:
# Write your code here
def unigram_lm(corpus, vocabulary):
    corpus = re.sub(r'[\s]+', ' ', corpus)
    tokenized_corpus = tokenize_bpe(corpus, merge_ops)
    unigram_probs = {}
    for word in vocabulary:
        unigram_probs[word] =  tokenized_corpus.count(word) / len(tokenized_corpus)
    
    ...
    return unigram_probs


In [46]:
unigram_probs = unigram_lm(corpus, vocabulary)
unigram_probs


{'B': 0.0006136668088577565,
 'é': 7.483741571436055e-05,
 'K': 0.00026193095500026193,
 'henn': 0.00020206102242877348,
 'T': 0.00022451224714308164,
 'on': 0.0044004400440044,
 ';': 2.2451224714308165e-05,
 'g': 0.017579308951303295,
 'ar': 0.007349034223150207,
 'b': 0.01142018963801142,
 'or': 0.005560419987576989,
 'M': 0.0018559679097161416,
 '.': 0.016756097378445328,
 '9': 1.496748314287211e-05,
 'll': 0.0045351473922902496,
 'och': 0.00811985960500812,
 'ed': 0.004145992830575575,
 'å': 0.01603017444601603,
 'P': 0.00017960979771446532,
 'r': 0.023132245197308846,
 'V': 0.0003966383032861109,
 'Å': 0.0003442521122860585,
 'am': 0.0035547772464321263,
 '8': 7.483741571436055e-06,
 '’': 7.483741571436055e-06,
 'd': 0.021074216265163932,
 'O': 0.0012946872918584375,
 'y': 0.005478098830291192,
 'enn': 0.00037418707857180276,
 'ng': 0.005567903729148425,
 'är': 0.006129184347006129,
 'a': 0.031102429970888246,
 '!': 0.0004714757190004715,
 'ill': 0.00411605786428983,
 'än': 0.0043

In [47]:
len(unigram_probs)


117

### Unigram Tokenization

You will now apply your unigram language model to tokenize a character sequence that does not include spaces, typically a single word in the Latin or Greek scripts or a sequence of words in Asian scripts, like Chinese or Korean.

Write a `tokenize_lm()` function that takes a character sequence, `char_seq`, and a dictionary of unigram probabilities, `unigram_probs`,  as input and returns the subword tokens and the segmentation probability, (prob,tokens). You will only return the token list with the highest probability.

    def tokenize_lm(char_seq, unigram_probs):

      ...

      return max(candidates)

As an example, applying 

`tokenize_lm('senare', unigram_probs)`
results in

`(2.0899522820189735e-07, ['s', 'en', 'ar', 'e'])`

Your function will cache (memoize) the results to speed up the computation. It will be similar to that of Norvig's in the notebook: How to Do Things with Words.ipynb. You can reuse it.
Python has a built-in memoization function that you can use: @functools.lru_cache(maxsize=2**10). You can also use the newer @functools.cache() function if you have Python 3.9 or higher. See here: https://docs.python.org/3/library/functools.html.

In [48]:
import functools


def tokenize_lm(char_seq, unigram_probs):
    # Use one of the two cache functions below to have a faster answer:
    # @functools.lru_cache(maxsize=2**10)
    @functools.cache  # Available from Python 3.9
    def __tokenize_lm(char_seq):
        word = char_seq
        word_prob = unigram_probs.get(word, 0)
        if len(char_seq) == 1:
            return (word_prob, [word])
        splits = [(word[:i], word[i:]) for i in range(1, len(word))]
        splits_tok = [(__tokenize_lm(split[0]), __tokenize_lm(split[1])) for split in splits]
        splits_tok = [(split[0][0] * split[1][0], split[0][1] + split[1][1]) for split in splits_tok]
        splits_tok.append((word_prob, [word]))
        return max(splits_tok, key=lambda x: x[0])
                
        # Write your code here
        ...

    return __tokenize_lm(char_seq)


In [49]:
tokenize_lm('senare', unigram_probs)


(8.533437392563631e-08, ['s', 'en', 'ar', 'e'])

### Text Tokenization with Unigrams

The previous function applies to a sequence without spaces. You will now apply it to your corpus. Write a `tokenize_text_lm()` function that takes the whole `corpus` string as input and the unigram probabilities `unigram_probs` and return the corpus probability and the tokenized subwords. 

This function is just an application of the functions you just wrote, where you will:
1. `str.split()` the string by whitespaces
2. Break the tokens into subtokens using `tokenize_lm()`. This function will return the probabilities of the resulting sequences;
3. Sum the logarithm of these probabilities. Use log10 to check your output with the numbers in the notebook. 

It is very significant that you use the logarithm of the probabilities and the sum. If you multiply the probabilities, you will get an underflow.

In [50]:
# Write your code
def tokenize_text_lm(corpus, unigram_probs):
    tokenized_corpus = []
    corpus_prob = 0.0
    for word in corpus.split():
        word_prob, word_tok = tokenize_lm(word, unigram_probs)
        corpus_prob += math.log10(word_prob)
        word_tok = ["_" + word_tok[0]] + word_tok[1:]
        tokenized_corpus += word_tok
    ...
    return corpus_prob, tokenized_corpus


In [51]:
init_loglikelihood, tokens = tokenize_text_lm(corpus, unigram_probs)


In [52]:
init_loglikelihood, tokens[:10]


(-195827.70286482148, ['_S', 'e', 'l', 'm', 'a', '_L', 'ag', 'er', 'l', 'ö'])

### Vocabulary Selection

You will now implement the final loop, where you will, at each iteration:
1. Select one subword and remove it from your vocabulary.
2. Estimate the probabilities of the subwords of the reduced vocabulary using `unigram_lm()`
2. Compute the resulting log-likelihood of the corpus without this word. You will use `tokenize_text_lm()` this time
3. Compute the loss, i.e. the log-likelihood reduction when the subword is removed from the current vocabulary

You will always keep the single characters in your vocabulary to avoid unknown words.

Store the pairs, (log-likelihood, removed_subword) in a list `logloss_word` and rank them by likelihood value.

In [53]:
logloss_word = []

In [54]:
# To have the same results
vocabulary_sorted = sorted(vocabulary, key=lambda w: -unigram_probs[w])
vocabulary_sorted[:10]


['t', 'a', 's', 'l', 'r', 'e', 'd', ',', 'k', 'm']

In [55]:
# Write your code here
for word in tqdm.tqdm(vocabulary_sorted):
    if len(word) == 1:
        continue
    temp_vocabulary = vocabulary.copy()
    temp_vocabulary.remove(word)
    unigram_probs_temp = unigram_lm(corpus, temp_vocabulary)
    logloss, tokens = tokenize_text_lm(corpus, unigram_probs_temp)
    logloss_word.append((logloss, word))


100%|██████████| 117/117 [02:36<00:00,  1.34s/it]


In [56]:
logloss_word_1 = list(map(lambda x: ((x[0]), x[1]), logloss_word))
sorted(logloss_word_1, reverse=True)


[(-195827.70286482148, 'henn'),
 (-195830.58202852483, 'enn'),
 (-196100.71907599052, 'am'),
 (-196203.833641265, 'tt'),
 (-196247.75652769106, 'ag'),
 (-196256.90154455567, 'li'),
 (-196260.5101205949, 'fr'),
 (-196295.26475448476, 'id'),
 (-196303.93999082092, 'ch'),
 (-196319.57007291287, 'ör'),
 (-196337.33252266241, 'll'),
 (-196351.2058036365, 'ed'),
 (-196360.55232036576, 'all'),
 (-196372.41403659558, 'den'),
 (-196408.1472140839, 'ing'),
 (-196418.92939948797, 'så'),
 (-196460.7473209377, 'ng'),
 (-196473.70795922022, 'hade'),
 (-196497.3970114398, 'na'),
 (-196510.03024404126, 'som'),
 (-196529.69035139968, 'det'),
 (-196548.7750217622, 'un'),
 (-196572.51740410086, 'på'),
 (-196575.2855887262, 'in'),
 (-196617.63144044214, 'fv'),
 (-196652.47750911533, 'on'),
 (-196676.91366302836, 'ej'),
 (-196701.68579294777, 'ar'),
 (-196702.65098805178, 'än'),
 (-196735.36924042017, 'sk'),
 (-196749.80994154414, 'ade'),
 (-196772.36521981275, 'ill'),
 (-196774.958114924, 'et'),
 (-196795

You will reduce now your vocabulary by one token: `out_candidate`. Write the piece of code to determine it.

In [57]:
# Write your code here
out_candidate = min(logloss_word, key=lambda x: x[0])


In [58]:
out_candidate


(-200062.3987292016, 'en')

## The tokenization without the subword

In [72]:
vocabulary_sorted.remove('ta')

ValueError: list.remove(x): x not in list

In [66]:
unigram_probs = unigram_lm(corpus, vocabulary_sorted)


In [67]:
new_loglikelihood, tokens = tokenize_text_lm(corpus, unigram_probs)


In [68]:
init_loglikelihood

-195827.70286482148

In [69]:
new_loglikelihood


-200062.3987292016

In [71]:
tokens[:10]


['_S', 'e', 'l', 'm', 'a', '_L', 'ag', 'er', 'l', 'ö']

If you are interested, you can improve this program and test it on larger corpora. You can also read a fine implementation of BPE by Andrej Karpathy: https://github.com/karpathy/minGPT/blob/master/mingpt/bpe.py

## Turning in your assignment

Now your are done with the program. To complete this assignment, you will write a report where you will:
1. Describe the background as well as the algorithms you used. For this, summarize the articles as described in the notebook:
   * Preliminaries: subword tokenizers
   * Design of the BPE Algorithm
   * Unigram Language Model
2. Describe your program as well as your results

The whole report should be of 2 to 3 pages.

Submit your report as well as your **notebook** (for archiving purposes) to Canvas: https://canvas.education.lu.se/. To write your report, you can either
1. Write directly your text in Canvas, or
2. Use Latex and Overleaf (www.overleaf.com). This will probably help you structure your text. You will then upload a PDF file in Canvas.

The submission deadline is September 29, 2023.

## Curious?

If you are interested, you can read the sentencepiece implementation by the author:
https://github.com/google/sentencepiece/tree/master

Concerning BPE, you can also read a fine implementation of BPE by Andrej Karpathy: https://github.com/karpathy/minGPT/blob/master/mingpt/bpe.py